In [1]:
import requests
import pandas as pd
import random
import time

CVR_VIRK_URL = 'http://distribution.virk.dk/cvr-permanent/virksomhed/_search'
CVR_PROD_URL = 'http://distribution.virk.dk/cvr-permanent/produktionsenhed/_search'
CVR_PROD_SCROLL = 'http://distribution.virk.dk/cvr-permanent/produktionsenhed/_search?scroll=1m'
CVR_SCROLL = 'http://distribution.virk.dk/_search/scroll'

In [4]:
auth_bruger = ''
auth_pass = ''

auth = (auth_bruger, auth_pass)

In [15]:
HEADERS = {
    'Content-Type': 'application/json',
}

FIELDS = ["Vrvirksomhed.cvrNummer",
          "Vrvirksomhed.deltagerRelation"]

In [85]:
def get_orgs(cvr, auth, fields = FIELDS, api_endpoint = CVR_VIRK_URL, headers = HEADERS):
    query_body = {"_source": fields,
                  "query": {
                      "bool": {
                          "must": [
                              {"match": {"Vrvirksomhed.cvrNummer": cvr}}
                          ]
                      }
                  }
                 }
    
    
    query = str(query_body).replace('\'', '\"')
    
    response = requests.post(api_endpoint, headers = headers, auth = auth, data = query)
    
    data_response = dict(response.json())

    #data = pd.json_normalize(data_response['hits']['hits'])
    
    return(data_response)

In [86]:
orgs = get_orgs("10010861", auth = auth).get('hits').get('hits')[0].get('_source').get('Vrvirksomhed').get('deltagerRelation')

In [87]:
revifirms = []

for org in orgs:
    if 'REVISION' in [orgnavn.get('hovedtype') for orgnavn in org.get('organisationer')]:
        revifirms.append(org)

In [88]:
revifirms[0].get('deltager').get('forretningsnoegle')

10092159

In [71]:
[[orgnavn.get('hovedtype') for orgnavn in org.get('organisationer')] for org in orgs]

[['STIFTERE', 'LEDELSESORGAN'],
 ['STIFTERE', 'LEDELSESORGAN', 'REGISTER', 'REGISTER'],
 ['LEDELSESORGAN', 'REGISTER'],
 ['REVISION'],
 ['REVISION'],
 ['REVISION'],
 ['REGISTER'],
 ['REGISTER'],
 ['REGISTER'],
 ['REGISTER']]

In [81]:
revifirms

[{'organisationer': [{'organisationsNavn': [{'sidstOpdateret': '2015-02-10T00:00:00.000+01:00',
      'navn': 'Revision',
      'periode': {'gyldigFra': '1999-11-01', 'gyldigTil': None}}],
    'enhedsNummerOrganisation': 4004733676,
    'hovedtype': 'REVISION',
    'medlemsData': [{'attributter': [{'type': 'FUNKTION',
        'vaerdier': [{'sidstOpdateret': '2016-05-27T13:14:12.000+02:00',
          'vaerdi': 'REVISION',
          'periode': {'gyldigFra': '2016-05-27', 'gyldigTil': None}}],
        'sekvensnr': 0,
        'vaerditype': 'string'}]}],
    'attributter': [{'type': 'FUNKTION',
      'vaerdier': [{'sidstOpdateret': '2015-02-10T00:00:00.000+01:00',
        'vaerdi': 'Revision',
        'periode': {'gyldigFra': '1999-11-01', 'gyldigTil': None}}],
      'sekvensnr': 0,
      'vaerditype': 'string'}]}],
  'kontorsteder': [{'penhed': {'navne': [{'sidstOpdateret': '2015-03-12T17:41:13.000+01:00',
       'navn': 'REVISIONSFIRMAET LEIF VESTERLUND - REGISTRERET REVISIONSANPARTSSELSK

In [69]:
[[orgnavn for orgnavn in org.get('organisationer') if 'REVISION' in orgnavn.get('hovedtype')] for org in orgs]

[[],
 [],
 [],
 [{'organisationsNavn': [{'sidstOpdateret': '2015-02-10T00:00:00.000+01:00',
     'navn': 'Revision',
     'periode': {'gyldigFra': '1999-11-01', 'gyldigTil': None}}],
   'enhedsNummerOrganisation': 4004733676,
   'hovedtype': 'REVISION',
   'medlemsData': [{'attributter': [{'type': 'FUNKTION',
       'vaerdier': [{'sidstOpdateret': '2016-05-27T13:14:12.000+02:00',
         'vaerdi': 'REVISION',
         'periode': {'gyldigFra': '2016-05-27', 'gyldigTil': None}}],
       'sekvensnr': 0,
       'vaerditype': 'string'}]}],
   'attributter': [{'type': 'FUNKTION',
     'vaerdier': [{'sidstOpdateret': '2015-02-10T00:00:00.000+01:00',
       'vaerdi': 'Revision',
       'periode': {'gyldigFra': '1999-11-01', 'gyldigTil': None}}],
     'sekvensnr': 0,
     'vaerditype': 'string'}]}],
 [{'organisationsNavn': [{'sidstOpdateret': '2015-02-10T00:00:00.000+01:00',
     'navn': 'Revision',
     'periode': {'gyldigFra': '1999-11-01', 'gyldigTil': None}}],
   'enhedsNummerOrganisation'

In [64]:
[org.get('organisationer') for org in orgs][0]

[{'organisationsNavn': [{'sidstOpdateret': '2015-02-10T00:00:00.000+01:00',
    'navn': 'Stiftere',
    'periode': {'gyldigFra': '1999-11-01', 'gyldigTil': None}}],
  'enhedsNummerOrganisation': 4004733674,
  'hovedtype': 'STIFTERE',
  'medlemsData': [{'attributter': [{'type': 'FUNKTION',
      'vaerdier': [{'sidstOpdateret': '2015-02-10T00:00:00.000+01:00',
        'vaerdi': 'STIFTERE',
        'periode': {'gyldigFra': '1999-11-01', 'gyldigTil': None}}],
      'sekvensnr': 0,
      'vaerditype': 'string'}]}],
  'attributter': [{'type': 'FUNKTION',
    'vaerdier': [{'sidstOpdateret': '2015-02-10T00:00:00.000+01:00',
      'vaerdi': 'Stiftere',
      'periode': {'gyldigFra': '1999-11-01', 'gyldigTil': None}}],
    'sekvensnr': 0,
    'vaerditype': 'string'}]},
 {'organisationsNavn': [{'sidstOpdateret': '2015-02-10T00:00:00.000+01:00',
    'navn': 'Direktion',
    'periode': {'gyldigFra': '1999-11-01', 'gyldigTil': None}}],
  'enhedsNummerOrganisation': 4004733675,
  'hovedtype': 'LEDELS